In [1]:
import pandas as pd
import os
import warnings # 用于发出警告信息

# --- 配置参数 ---

# 1. 包含各个 stage 子目录的基础路径
#    假设你的脚本和 'stage_specific_hvg_analysis' 目录在同一级
#    或者提供绝对路径 '/path/to/your/stage_specific_hvg_analysis'
BASE_OUTPUT_DIR = "D:/111/stage_specific_hvg_analysis"

# 2. 需要处理的 stage 名称列表
STAGES_TO_PROCESS = ['stage0', 'stage1', 'stage2', 'stage3', 'stage4', 'ALL']
#    注意：'ALL' 可能是你对所有阶段合并分析的结果目录名，请根据实际情况修改

# 3. 需要提取数据的目标列名（交集名称）
TARGET_INTERSECTIONS = [
    'M-MG & R-MG & S-MG',                     # 三种 MG 的交集
    'M-MG & R-AG & R-CG & R-MG & S-AG & S-MG', # 六个样本的总交集 (请确认此列名与你CSV中的完全一致)
    'R-AG & S-AG'                              # 两种 AG 的交集
]

# 4. 输出整合后的CSV文件名
CONSOLIDATED_OUTPUT_FILE = "consolidated_target_novel_genes.csv"

# --- 脚本执行 ---

# 用于存储最终结果的列表，每个元素是一个包含 [stage, intersection, gene] 的列表
consolidated_data = []

print("开始处理各个 stage 的 novel gene intersection 文件...")

# 遍历每个指定的 stage
for stage_name in STAGES_TO_PROCESS:
    print(f"\n--- 正在处理 Stage: {stage_name} ---")

    # 构建当前 stage 的子目录路径
    stage_dir = os.path.join(BASE_OUTPUT_DIR, stage_name)

    # 构建目标 CSV 文件的完整路径
    # 文件名格式根据你的描述是 'Stage_{stage_name}_HVGs_intersection_novel_gene_overlap.csv'
    csv_filename = f'Stage_{stage_name}_HVGs_intersection_novel_gene_overlap.csv'
    csv_filepath = os.path.join(stage_dir, csv_filename)

    # 检查目标 CSV 文件是否存在
    if not os.path.exists(csv_filepath):
        warnings.warn(f"文件未找到，跳过: {csv_filepath}")
        continue # 跳到下一个 stage

    # 读取 CSV 文件
    try:
        df = pd.read_csv(csv_filepath)
        print(f"  成功读取文件: {csv_filename}")

        # 遍历需要提取的目标交集列名
        for intersection_col in TARGET_INTERSECTIONS:
            # 检查目标列是否存在于当前 DataFrame 中
            if intersection_col in df.columns:
                print(f"    找到目标列: '{intersection_col}'")
                # 提取该列的基因名
                # 使用 .dropna() 去除因为列长不一致而填充的 NaN/空值
                # 使用 .astype(str) 确保基因名是字符串
                # 使用 .unique() 可选，如果原始文件可能有重复基因且你只想保留唯一值
                genes_in_intersection = df[intersection_col].dropna().astype(str).tolist()

                if genes_in_intersection:
                    print(f"      提取到 {len(genes_in_intersection)} 个基因.")
                    # 将每个基因添加到结果列表中
                    for gene in genes_in_intersection:
                        # 确保基因名不是空字符串（虽然dropna通常能处理，加一层保险）
                        if gene.strip():
                            consolidated_data.append([stage_name, intersection_col, gene.strip()])
                else:
                    print(f"      列 '{intersection_col}' 中没有找到有效基因名。")

            else:
                # 如果目标列不存在，发出警告
                warnings.warn(f"在文件 {csv_filename} 中未找到列: '{intersection_col}'")

    except Exception as e:
        # 处理读取文件时可能发生的其他错误
        warnings.warn(f"读取文件时发生错误 {csv_filepath}: {e}")
        continue # 跳到下一个 stage

# 将收集到的数据转换为 DataFrame
if consolidated_data:
    consolidated_df = pd.DataFrame(consolidated_data, columns=['stage', 'intersection', 'gene'])
    print(f"\n数据整合完成，共收集到 {len(consolidated_df)} 条记录。")

    # 保存 DataFrame 到 CSV 文件
    try:
        consolidated_df.to_csv(CONSOLIDATED_OUTPUT_FILE, index=False)
        print(f"结果已成功保存到: {CONSOLIDATED_OUTPUT_FILE}")
    except Exception as e:
        print(f"保存最终CSV文件时出错: {e}")

else:
    print("\n未能收集到任何数据，未生成输出文件。请检查输入目录、文件名和列名是否正确。")

print("\n脚本执行完毕。")

开始处理各个 stage 的 novel gene intersection 文件...

--- 正在处理 Stage: stage0 ---
  成功读取文件: Stage_stage0_HVGs_intersection_novel_gene_overlap.csv
    找到目标列: 'M-MG & R-MG & S-MG'
      提取到 22 个基因.
    找到目标列: 'M-MG & R-AG & R-CG & R-MG & S-AG & S-MG'
      提取到 9 个基因.
    找到目标列: 'R-AG & S-AG'
      提取到 34 个基因.

--- 正在处理 Stage: stage1 ---
  成功读取文件: Stage_stage1_HVGs_intersection_novel_gene_overlap.csv
    找到目标列: 'M-MG & R-MG & S-MG'
      提取到 18 个基因.
    找到目标列: 'M-MG & R-AG & R-CG & R-MG & S-AG & S-MG'
      提取到 8 个基因.
    找到目标列: 'R-AG & S-AG'
      提取到 62 个基因.

--- 正在处理 Stage: stage2 ---
  成功读取文件: Stage_stage2_HVGs_intersection_novel_gene_overlap.csv
    找到目标列: 'M-MG & R-MG & S-MG'
      提取到 19 个基因.
    找到目标列: 'M-MG & R-AG & R-CG & R-MG & S-AG & S-MG'
      提取到 7 个基因.
    找到目标列: 'R-AG & S-AG'
      提取到 52 个基因.

--- 正在处理 Stage: stage3 ---
  成功读取文件: Stage_stage3_HVGs_intersection_novel_gene_overlap.csv
    找到目标列: 'M-MG & R-MG & S-MG'
      提取到 26 个基因.
    找到目标列: 'M-MG & R-AG & R-CG & R-MG & S-AG & S